In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

plt.style.use('default') 


plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
sns.set(font_scale = 1)

In [2]:
labels = pd.read_csv('train_labels.csv')
values = pd.read_csv('train_values.csv')
values = values.merge(labels)

## ¿Cuales fueron las regiones geograficas mas y menos dañadas?

Para responder esta pregunta utilizamos dos criterios, que se diferencian en lo que se entiende por region geografica. Para el primer caso tomamos como de una misma region a todos los que comparten el geo_level_1_id y para el segundo a todos los que comparten los 3 ids de la region. En ambos casos para realizar el analisis nos quedamos solo con las regiones que contenian al menos una cierta cantidad de edificaciones, para disminuir el desvio standar de la muestra. 

Para los dos casos el dato que evaluamos fue el grado de daño (coinciderandolo como un numero) y nos quedamos con los que tenian un promedio mayor o igual 2.75 (lo que querria decir que los daños en la zona fueron severos) o menor o igual a 1.25 (lo que querria decir que la zona no fue tan impactada) y una desviacion standar menor o igual 0.5, para garantizar que el promedio sea representativo de las edificaciones que estan en la zona. Para el caso en el que definimos una zona como los que comparten el geo_level_1_id nos quedamos con las zonas de al menos 5000 edificaciones y para el otro caso nos quedamos con las zonas de al menos 100 edificaciones.

In [4]:
aux = values.groupby('geo_level_1_id')['damage_grade']\
    .agg(['count', 'std', 'mean'])
aux.loc[(aux['count']>=5000) & (aux['std']<= 0.5) & 
      ((aux['mean'] >= 2.75) |  (aux['mean']<= 1.25))]

,count,std,mean
geo_level_1_id,,,
17,21813,0.435225,2.79448


In [5]:
aux = values.groupby(['geo_level_1_id', 'geo_level_2_id',
                    'geo_level_3_id'])['damage_grade']\
    .agg(['count', 'std', 'mean'])
aux.loc[(aux['count']>=100) & (aux['std']<= 0.5) & 
      ((aux['mean'] >= 2.75) |  (aux['mean']<= 1.25))]

count       std      mean
geo_level_1_id geo_level_2_id geo_level_3_id                           
3              720            6460              107  0.415491  2.841121
               1006           5957              120  0.222035  2.966667
6              1253           3073              172  0.404987  1.186047
10             87             2755              117  0.447510  2.820513
17             421            4900              127  0.379614  2.850394
               682            1743              117  0.130179  2.982906
               1080           4391              143  0.478536  2.804196
                              6609              111  0.413530  2.783784
21             363            3485              126  0.432417  2.753968
                              4982              107  0.443810  2.803738
                              12093             111  0.378367  2.828829
26             39             1324              100  0.368453  1.160000
27             1155           7851              108  0.135445  2.981481

## ¿Existen combinaciones de valores para las cuales los daños hayan sido muy altos o muy bajos?

Para responder esto, agrupamos las variables de a pares y de todas las combinaciones de valores posibles nos quedamos con las que tienen en promedio daños mayores o iguales a 2.75 o menores iguales a 1.25. Para evitar que el valor elevado o bajo del promedio sea concecuencia de que nos estamos quedando con pocas edificaciones, nos quedamos solo con las combinaciones de al menos 3000 edificaciones.

In [6]:
cols = values.columns
resultados = []

for i in range(1,len(cols)-1):
    # Lo hago en ese rango para no usar ni bulding_id
    # ni el damage_grade
    for j in range(i+1,len(cols)-1):
        aux = values[[cols[i], cols[j], 'damage_grade']]
        aux = aux.groupby([cols[i],cols[j]]).agg(['mean', 'count'])
        aux = aux.loc[(aux[('damage_grade', 'count')]>3000) &
        ((aux[('damage_grade', 'mean')]>=2.75)|
        (aux[('damage_grade', 'mean')]<=1.25))]
        if (len (aux)>0):
            resultados.append(aux.index)

resultados

[MultiIndex([(17, 2),
             (17, 3)],
            names=['geo_level_1_id', 'count_floors_pre_eq']),
 MultiIndex([(17, 10),
             (17, 15)],
            names=['geo_level_1_id', 'age']),
 MultiIndex([(17, 6),
             (17, 7)],
            names=['geo_level_1_id', 'area_percentage']),
 MultiIndex([(17, 4),
             (17, 5),
             (17, 6),
             (17, 7)],
            names=['geo_level_1_id', 'height_percentage']),
 MultiIndex([(17, 't')],
            names=['geo_level_1_id', 'land_surface_condition']),
 MultiIndex([(17, 'r')],
            names=['geo_level_1_id', 'foundation_type']),
 MultiIndex([(17, 'n'),
             (17, 'q')],
            names=['geo_level_1_id', 'roof_type']),
 MultiIndex([(17, 'f')],
            names=['geo_level_1_id', 'ground_floor_type']),
 MultiIndex([(17, 'q'),
             (17, 'x')],
            names=['geo_level_1_id', 'other_floor_type']),
 MultiIndex([(17, 's'),
             (17, 't')],
            names=['geo_level_1_

Dado que todos los resultados obtenidos contienen a la region 17, y como vimos anteriormente esta recibio daños muy severos, se puede pensar que los resultados tienen mas que ver con dicha region que con la combinacion de valores en si.

## ¿Existen combinaciones de valores para las cuales todas las edificaciones tengan grados de daño parecidos?

Al igual que en el caso anterior, agrupamos las variables de a pares pero esta vez nos quedamos solo con las que tenian al menos 2000 edificaciones y una desviacion estandard menor o igual a 0.35

In [7]:
cols = values.columns
resultados = []
for i in range(1,len(cols)-1):
# Voy en ese rango para que no tome builiding_id ni damage_grade
    for j in range(i+1,len(cols)-1):
        aux = values[[cols[i], cols[j], 'damage_grade']]
        aux = aux.groupby([cols[i],cols[j]]).agg(['std', 'count'])
        aux = aux.loc[(aux[('damage_grade', 'count')]>=2000) &
        ((aux[('damage_grade', 'std')]<= 0.35))]
        if (len (aux)>0):
            resultados.append(aux.index)

resultados

[MultiIndex([(17, 25)],
            names=['geo_level_1_id', 'age']),
 MultiIndex([(17, 'q')],
            names=['geo_level_1_id', 'roof_type'])]

Como en el caso anterior, todas las combinaciones contienen a la region 17, por lo que tendemos a creer que son concecuencia mas de esa region que de la combinacion de valores en si.